In [ ]:
"""
Purpose: To check on the status of the Allen split suggestions pipeline
as they run through

- Downloading
- Decimation
- Soma Finding
- Decomposition
- Suggestion Splits

"""

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [4]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-03-05 18:06:55,641 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:55,643 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:55,644 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:55,648 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-05 18:06:55,649 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:55,663 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-05 18:06:56,060 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:56,064 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:56,065 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:56,066 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:56,067 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:56,070 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:56,071 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:56,072 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:56,492 - settings - Setting enable_python_native_blobs to True


In [5]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-05 18:06:56,649 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 18:06:56,651 - settings - Setting database.user to celiib
INFO - 2021-03-05 18:06:56,652 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 18:06:56,656 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 18:06:57,113 - settings - Setting enable_python_native_blobs to True


# Investigating why some neurons did not get split suggestions

In [80]:
allen_table = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021))
allen_table

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691134988608634,3,3,2021,2
864691135065024068,3,3,2021,2
864691135155711844,3,3,2021,2
864691135210232512,3,3,2021,2
864691135341057989,3,3,2021,2
864691135345291423,3,3,2021,2
864691135373423816,3,3,2021,2
864691135373676873,3,3,2021,2
864691135382990571,3,3,2021,2
864691135463533125,3,3,2021,2


In [83]:
minnie.BaylorSegmentCentroid() & allen_table  & dict(decimation_ratio=0.25) & "multiplicity=0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691135564804951,0,0.25,0,30.00,None,None,None,0,0,=BLOB=,0,nan,nan,nan,nan,nan,1323.0784113407135
864691136922748900,0,0.25,0,30.00,None,None,None,0,0,=BLOB=,0,nan,nan,nan,nan,nan,597.8733191490173


In [85]:
nviz.plot_objects(du.fetch_segment_id_mesh(864691136922748900))

In [89]:
key_source = minnie.Decomposition() & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988608634,0,0.25,30.00,5,0,1,=BLOB=,1150159,2311814,0,1,1,1,2,11,346,233,11067259.173807988,5580597.123682757,15796.577,104.973,104.973,450.146,379.101,4184,0.00037805204832484123,12.092485549132949,6872596.828,156,0.0006087946237372387,26.821,946998934578.9817,136631721.2458318,85567.61161066596,137793.46559669625,6070505990.890908,4574.8436
864691135065024068,0,0.25,30.00,5,0,1,=BLOB=,1683573,3378174,0,2,2,0,1,11,346,101,15448645.134033982,4735340.661534153,28195.927,309.528,279.925,621.111,612.357,4401,0.00028487935102505666,12.719653179190752,12951735.977,254,0.00033980000888030764,17.327,644008505500.065,97079430.94166055,41687.054101675785,49723.72094703834,2535466557.0868697,7567.1259
864691135155711844,0,0.25,30.00,6,0,1,=BLOB=,1323348,2681606,0,2,2,0,1,12,247,99,10335501.970505765,4157038.1039918233,26480.998,386.808,304.693,553.981,502.772,6481,0.0006270619480790302,26.238866396761132,8914214.834,199,0.0007270410373419097,32.568,1301368737143.5571,126957093.46514548,125912.48503045616,145988.0383609293,6539541392.681191,3068.9409
864691135210232512,0,0.25,30.00,6,0,1,=BLOB=,1104606,2279207,0,5,4,2,2,15,374,97,11682255.066969572,3937393.8053746116,18011.375,128.134,128.134,454.419,431.091,2282,0.00019533899807171054,6.101604278074866,7371490.013,204,0.0003095710631060445,11.186,390388762128.02075,108385813.17371431,33417.24349366473,52959.27437187735,1913670402.588337,2271.6559
864691135341057989,0,0.25,30.00,6,0,1,=BLOB=,910711,1829521,0,3,1,2,2,14,259,112,10163119.200359212,4474894.858952958,28470.197,143.696,143.696,469.631,412.785,3107,0.0003057132302344918,11.996138996138995,6203039.418,137,0.0005008834847936154,22.679,557754542849.0073,106585138.54347737,54880.25200268178,89916.3305702223,4071201042.6934843,2310.7131
864691135345291423,0,0.25,30.00,4,0,1,=BLOB=,708043,1432111,0,0,0,0,1,6,249,195,9952249.61378499,7106458.13114

In [48]:
key_source & "n_multi_soma_touching_limbs = 0 AND n_same_soma_multi_touching_limbs=0 "

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135345291423,0,0.25,30.00,4,0,1,=BLOB=,708043,1432111,0,0,0,0,1,6,249,195,9952249.61378499,7106458.131142457,25318.306,75.818,75.818,386.718,341.864,1532,0.00015393504578884427,6.152610441767068,3636560.395,70,0.00042127720526967903,21.886,366595548966.5923,173941231.56845084,36835.44557190528,100808.3213661552,5237079270.951319,3031.7047
864691135373423816,0,0.25,30.00,4,0,1,=BLOB=,3383405,6837747,0,0,0,0,1,8,853,808,29828707.39402775,27323282.569861326,25105.522,83.504,83.504,231.078,231.078,359,1.2035385752983661e-05,0.4208675263774912,8650394.095,265,4.1500999383080705e-05,1.355,59226635728.49248,77291152.34234586,1985.5582391193634,6846.697974457136,223496738.5980848,8559.7473
864691135753669837,0,0.25,30.00,6,0,1,=BLOB=,911257,1828582,0,0,0,0,1,7,223,125,9555667.06060449,5075904.344280557,28775.272,138.908,138.908,443.566,421.249,3297,0.00034503085750995516,14.784753363228699,5739507.398,111,0.0005744395418235508,29.703,590011800042.7266,95186744.61625417,61744.70042758087,102798.33426964885,5315421622.006546,3114.6118
864691135782479155,0,0.25,30.00,6,0,1,=BLOB=,1047370,2136538,0,0,0,0,1,8,339,269,11323559.270325247,7629342.894595036,18731.911,176.216,176.216,610.191,587.55,2462,0.00021742280330990698,7.262536873156342,6799202.788,200,0.0003621012752179146,12.31,529217840349.0738,151800135.8102522,46735.997729614304,77835.27817159641,2646089201.745369,4909.5595
864691135939404545,0,0.25,30.00,3,0,1,=BLOB=,1055074,2118204,0,0,0,0,1,8,77,41,4121113.890599509,1977689.2964443937,36314.47,266.904,239.212,480.825,463.307,2035,0.0004937985345762826,26.428571428571427,3482205.607,58,0.0005843997252514906,35.086,465195474640.051,145079842.9413419,112881.0042598405,133592.19045104797,8020611631.725018,3277.444
864691135975576131,0,0.25,30.00,4,0,1,=BLOB=,3002136,6035385,0,0,0,0,1,7,958,902,32837367.129145086,29464143.240493502,25063.128,105.636,1

In [64]:
(minnie.Decomposition() & dict(segment_id=864691136008729518))

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135939404545,0,0.25,30.00,3,0,1,=BLOB=,1055074,2118204,0,0,0,0,1,8,77,41,4121113.890599509,1977689.2964443937,36314.47,266.904,239.212,480.825,463.307,2035,0.0004937985345762826,26.428571428571427,3482205.607,58,0.0005843997252514906,35.086,465195474640.051,145079842.9413419,112881.0042598405,133592.19045104797,8020611631.725018,3277.444


In [69]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=864691136008729518)).fetch1("decomposition")
neuron_obj

DataJointError: '/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691136008729518_0_25.pbz2' downloaded but did not pass checksum'

In [72]:
import datajoint_utils as du
neuron_obj = du.fetch_neuron_obj_manual(segment_id=864691136008729518)

Decompressing Neuron in minimal output mode...please wait


In [58]:
import neuron_utils as nru

In [60]:
segment_id = 864691135782479155
dec_mesh = du.fetch_segment_id_mesh(segment_id)
neuron_obj = nru.decompress_neuron("/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135782479155_0_25.pbz2",
                     original_mesh=dec_mesh)

Decompressing Neuron in minimal output mode...please wait


In [75]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [74]:
neuron_obj.n_error_limbs

0

In [73]:
nviz.visualize_neuron_lite(neuron_obj)


 Working on visualization type: mesh


In [ ]:
segment_id_with_no_errors = [
    864691135345291423, #had just soma on the very tip
    864691135373423816, #removed in glia
    864691135753669837,
    ,
]

In [45]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'") & "status='error'"
(curr_table).delete()# & "status='error'")

In [32]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
seg_ids_to_delete = [k["segment_id"] for k in curr_table.fetch("key")]
seg_ids_to_delete

[864691135490245479,
 864691135373676873,
 864691135474614848,
 864691135926327508,
 864691135815482447,
 864691135644807919,
 864691135576995845,
 864691137196949569]

In [33]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134219060901,0,0.25,30.00,0,0,1,=BLOB=,11723,24809,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.8309
864691134221889045,0,0.25,30.00,5,0,1,=BLOB=,12285,26788,0,1,1,0,1,1,3,3,7497.255110138911,7497.255110138911,942.191,298.182,298.182,1077.139,1077.139,1,0.00013338214924121366,0.3333333333333333,7497.255,3,0.00013338215120067278,0.333,55229792.033337906,55229792.033337906,7366.66836355187,7366.668471772389,18409930.677779302,16.8753
864691134242673681,0,0.25,30.00,0,0,1,=BLOB=,10252,20204,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.9755
864691134252908410,0,0.25,30.00,0,0,1,=BLOB=,10910,22764,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.2558
864691134316523453,0,0.25,30.00,0,0,1,=BLOB=,10031,22403,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5114
864691134384033872,0,0.25,30.00,0,0,1,=BLOB=,10015,20509,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8075
864691134391593458,0,0.25,30.00,0,0,1,=BLOB=,11947,25033,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.7341
864691134409056935,0,0.25,30.00,0,0,1,=BLOB=,10770,22630,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.9198
864691134428609977,0,0.25,30.00,0,0,1,=BLOB=,13062,28555,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.3236
864691134441691943,0,0.25,30.00,0,0,1,=BLOB=,10681,23432,0,0,0,0,1,0,0,None,0.0,nan,nan,nan,nan,nan,nan,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.9167


# All the cells that need to get suggestions

In [35]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988608634,0,0.25,30.00,5,0,1,=BLOB=,1150159,2311814,0,1,1,1,2,11,346,233,11067259.173807988,5580597.123682757,15796.577,104.973,104.973,450.146,379.101,4184,0.00037805204832484123,12.092485549132949,6872596.828,156,0.0006087946237372387,26.821,946998934578.9817,136631721.2458318,85567.61161066596,137793.46559669625,6070505990.890908,4574.8436
864691135065024068,0,0.25,30.00,5,0,1,=BLOB=,1683573,3378174,0,2,2,0,1,11,346,101,15448645.134033982,4735340.661534153,28195.927,309.528,279.925,621.111,612.357,4401,0.00028487935102505666,12.719653179190752,12951735.977,254,0.00033980000888030764,17.327,644008505500.065,97079430.94166055,41687.054101675785,49723.72094703834,2535466557.0868697,7567.1259
864691135155711844,0,0.25,30.00,6,0,1,=BLOB=,1323348,2681606,0,2,2,0,1,12,247,99,10335501.970505765,4157038.1039918233,26480.998,386.808,304.693,553.981,502.772,6481,0.0006270619480790302,26.238866396761132,8914214.834,199,0.0007270410373419097,32.568,1301368737143.5571,126957093.46514548,125912.48503045616,145988.0383609293,6539541392.681191,3068.9409
864691135210232512,0,0.25,30.00,6,0,1,=BLOB=,1104606,2279207,0,5,4,2,2,15,374,97,11682255.066969572,3937393.8053746116,18011.375,128.134,128.134,454.419,431.091,2282,0.00019533899807171054,6.101604278074866,7371490.013,204,0.0003095710631060445,11.186,390388762128.02075,108385813.17371431,33417.24349366473,52959.27437187735,1913670402.588337,2271.6559
864691135341057989,0,0.25,30.00,6,0,1,=BLOB=,910711,1829521,0,3,1,2,2,14,259,112,10163119.200359212,4474894.858952958,28470.197,143.696,143.696,469.631,412.785,3107,0.0003057132302344918,11.996138996138995,6203039.418,137,0.0005008834847936154,22.679,557754542849.0073,106585138.54347737,54880.25200268178,89916.3305702223,4071201042.6934843,2310.7131
864691135382990571,0,0.25,30.00,5,0,1,=BLOB=,915671,1894934,0,1,0,1,2,13,251,93,8502336.49131214,2099912.38933

In [8]:
no_splits = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)) - key_source.proj()
no_splits

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691135345291423,3,3,2021,2
864691135373423816,3,3,2021,2
864691135564804951,3,3,2021,2
864691135753669837,3,3,2021,2
864691135782479155,3,3,2021,2
864691135939404545,3,3,2021,2
864691135975576131,3,3,2021,2
864691136008578990,3,3,2021,2
864691136008729518,3,3,2021,2
864691136922748900,3,3,2021,2


In [ ]:
minnie.DecompositionSplit() & key_source.proj()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")

# Getting the split suggesstions

In [93]:
import datajoint as dj
dj.U("segment_id") & minnie.DecompositionSplit() & key_source.proj()

segment_id segment id unique within each Segmentation
864691134988608634
864691135065024068
864691135155711844
864691135210232512
864691135341057989
864691135373676873
864691135382990571
864691135463533125
864691135472075826
864691135474614848


In [94]:
du.create_suggested_splits_neuroglancer_spreadsheet?

In [95]:
allen_segment_ids = (minnie.AllenProofreading() & dict(day=3,month=3,year=2021)).fetch("segment_id")
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=allen_segment_ids,output_type="server",
                                                   print_links=True,
                                                   output_filename="allen_spreadsheet_3_10.csv")

  0%|          | 0/43 [00:00<?, ?it/s]

segment_id = 864691134988608634
segment_id = 864691135065024068
segment_id = 864691135155711844
segment_id = 864691135210232512
segment_id = 864691135341057989
segment_id = 864691135373676873
segment_id = 864691135382990571
segment_id = 864691135463533125
segment_id = 864691135472075826
segment_id = 864691135474614848
segment_id = 864691135490245479
segment_id = 864691135510515408
segment_id = 864691135571307557
segment_id = 864691135576995845
segment_id = 864691135609490439
segment_id = 864691135644807919
segment_id = 864691135653543364
segment_id = 864691135753592525
segment_id = 864691135758491982
segment_id = 864691135775809453
segment_id = 864691135807230237
segment_id = 864691135815482447
segment_id = 864691135841095907
segment_id = 864691135850464967
segment_id = 864691135850470343
segment_id = 864691135883914352
segment_id = 864691135926327508
segment_id = 864691135945417764
segment_id = 864691136008436140
segment_id = 864691136008496556
segment_id = 864691136008511148
segment_